In [1]:
from os import environ
environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.9.0 pyspark-shell' 

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ba20c5e9-53d5-4faa-88db-37b173a10505;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.9.0 in central
	found commons-io#commons-io;2.6 in central
	found org.glassfish.jaxb#txw2;2.3.2 in central
:: resolution report :: resolve 218ms :: artifacts dl 5ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.9.0 from central in [default]
	commons-io#commons-io;2.6 from central in [default]
	org.glassfish.jaxb#txw2;2.3.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search

In [3]:
XML_PATH = 'enwiki-20221001-pages-articles-multistream6.xml-p958046p1483661'

In [4]:
df = spark.read.format("com.databricks.spark.xml").option("rowTag", "page").load(XML_PATH)

In [5]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- ns: long (nullable = true)
 |-- redirect: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _title: string (nullable = true)
 |-- revision: struct (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _deleted: string (nullable = true)
 |    |-- contributor: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _deleted: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- ip: string (nullable = true)
 |    |    |-- username: string (nullable = true)
 |    |-- format: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- minor: string (nullable = true)
 |    |-- model: string (nullable = true)
 |    |-- parentid: long (nullable = true)
 |    |-- sha1: string (nullable = true)
 |    |-- text: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |  

In [6]:
from pyspark.sql.functions import col
df_body = df.select(col("revision.text._VALUE"), col("title"))
df_body.printSchema()

root
 |-- _VALUE: string (nullable = true)
 |-- title: string (nullable = true)



In [7]:
df_body.show()

+--------------------+--------------------+
|              _VALUE|               title|
+--------------------+--------------------+
|{{Infobox settlem...|       Morin-Heights|
|{{Short descripti...|           Getafe CF|
|{{Short descripti...|Human interface g...|
|#REDIRECT [[Coalg...|   Sweedler notation|
|#REDIRECT [[coalg...| Sweedler's notation|
|#REDIRECT [[coalg...|      Co-commutative|
|#REDIRECT [[Coalg...|       Cocommutative|
|'''HIG''' may ref...|                 HIG|
|{{wiktionary|prev...|       Prevarication|
|{{short descripti...|      Pietro Cataldi|
|#redirect [[Pietr...|Pietro Antonio Ca...|
|#REDIRECT [[Peace...| Peace of westphalia|
|'''Cataldi''' is ...|             Cataldi|
|{{Short descripti...|97th United State...|
|{{short descripti...|         Rick Hearst|
|#REDIRECT [[Legis...|  Ontario parliament|
|#redirect [[Thirt...|   Thirty year's war|
|#REDIRECT [[Legis...| Ontario legislature|
|#REDIRECT [[Flext...|           Flex time|
|#REDIRECT [[Excep...|       Err

In [8]:
def get_artist_expresions():
    musician_flags = []
    with open('musician_occupancy_flags', 'r', encoding='utf8') as f:
        while flag := f.readline().rstrip():
            musician_flags.append("(?is)\| *?occupation.*?"+ flag +".*?\n")
    return "|".join(musician_flags)

In [9]:
expresion = get_artist_expresions()
df_artists = df_body.filter(df_body['_VALUE'].rlike(expresion))

In [10]:
df_artists.show()

+--------------------+--------------------+
|              _VALUE|               title|
+--------------------+--------------------+
|{{Short descripti...|Theodore Arthur B...|
|{{short descripti...|James Duncan McGr...|
|{{short descripti...|         Denis Hayes|
|{{Short descripti...|  Ron Taylor (actor)|
|{{Short descripti...|         Yoko Tawada|
|{{short descripti...|        J. B. Stoner|
|{{Short descripti...|       Lewis Gilbert|
|{{short descripti...|   Tommy F. Robinson|
|{{Short descripti...|Pedro Flores (com...|
|{{short descripti...|     Joseph Babinski|
|{{short descripti...|            Li Yundi|
|{{Use dmy dates|d...|    Toshinobu Kubota|
|{{Short descripti...| Christmas Humphreys|
|{{Short descripti...|   Gwendolyn MacEwen|
|{{short descripti...|      Michiko Naruke|
|{{Short descripti...|           Kenji Ito|
|{{short descripti...|         Akari Kaida|
|{{short descripti...|    Noriyuki Iwadare|
|{{Short descripti...|     Grzegorz Turnau|
|{{Short descripti...|         R

In [11]:
from udf_infobox import parse_infobox
from udf_discography import parse_discography
from udf_awards import parse_awards

In [12]:
df_parsed_infobox = df_artists.withColumn("about", parse_infobox(col("_VALUE")))
df_parsed_infobox = df_parsed_infobox.withColumn("songs", parse_discography(col("_VALUE")))
df_parsed_infobox = df_parsed_infobox.withColumn("awards_list", parse_awards(col("_VALUE")))

In [13]:
df_parsed_infobox.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|              _VALUE|               title|               about|               songs|         awards_list|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|{{Short descripti...|Theodore Arthur B...|{'honorific-prefi...|                  []|                  []|
|{{short descripti...|James Duncan McGr...|{'honorific-prefi...|                  []|                  []|
|{{short descripti...|         Denis Hayes|{'name': 'Denis H...|                  []|                  []|
|{{Short descripti...|  Ron Taylor (actor)|{'name': 'Ron Tay...|                  []|                  []|
|{{Short descripti...|         Yoko Tawada|{'name': 'Yōko Ta...|                  []|                  []|
|{{short descripti...|        J. B. Stoner|{'name': 'J.B. St...|                  []|                  []|
|{{Short descripti...|       Lewis Gi

In [14]:
sc = spark._jsc.sc() 
n_workers =  len([executor.host() for executor in sc.statusTracker().getExecutorInfos() ]) -1

print(n_workers)

2


In [15]:
cols = ["title", "about", "songs", "awards_list"]
df = df_parsed_infobox.select(*cols)

In [17]:
df.write.json("full_test.json")